In [ ]:
import pandas as pd
import re
import cn2an


def c_df():
    
    # 讀取檔案、建立 dataframe 物件

    city_list = ['a', 'b', 'e', 'f', 'h']

    dataframe_list = []

    for i in range(len(city_list)):
        temp_df = pd.read_csv('land_data/' + city_list[i] + '_lvr_land_a.csv')
        dataframe_list.append(temp_df)
    
    df_a = dataframe_list[0].iloc[1: , :]
    df_b = dataframe_list[1].iloc[1: , :]
    df_e = dataframe_list[2].iloc[1: , :]
    df_f = dataframe_list[3].iloc[1: , :]
    df_h = dataframe_list[4].iloc[1: , :]
    

def m_df():
    
    # 合併 dataframe 物件
    df_all = pd.concat([df_a, df_b, df_e, df_f, df_h], axis=0, ignore_index=True)
    
def fil_df():
    
    # 篩選【 主要用途 】 為 【 住家用 】
    df_housing = df_all.query("主要用途 == '住家用'")
    
    # 篩選【 建物型態 】 為 【 住宅大樓 】
    df_hous_building = df_housing[df_housing['建物型態'].apply(lambda x: True if re.search('^住宅大樓', x) else False)]
       
    # 抓出剩餘的 index
    index_list = []
    for i in df_hous_building.index:
        index_list.append(i)
    
    # 將樓層中文轉成數字(int)
    for j in index_list:
        df_hous_building['總樓層數'][j] = cn2an.transform(df_hous_building['總樓層數'][j], "cn2an")
        df_hous_building['總樓層數'][j] = int(df_hous_building['總樓層數'][j].replace("層", ""))
        
    # 篩選總樓層數大於等於十三層的資料
    df_layer = df_hous_building.query("總樓層數 >= 13")

    # 抓出剩餘的 index
    index2_list = []
    for i in df_layer.index:
        index2_list.append(i)

    # 將樓層從數字再轉回中文小寫
    for j in index2_list:
        df_layer['總樓層數'][j] = cn2an.an2cn(df_layer['總樓層數'][j], "low")
        df_layer['總樓層數'][j] = str(df_layer['總樓層數'][j]) + "層"
        
def output_csv():
    df_layer.to_csv('filter_a.csv')
    

if __name__ == "__main__":
    c_df()
    m_df()
    fil_df()
    output_csv()

In [1]:
import pandas as pd
import re
import cn2an

def c_df():
    
    # 讀取檔案、建立 dataframe 物件

    city_list = ['a', 'b', 'e', 'f', 'h']

    dataframe_list = []

    for i in range(len(city_list)):
        temp_df = pd.read_csv('land_data/' + city_list[i] + '_lvr_land_a.csv')
        dataframe_list.append(temp_df)
    
    df_a = dataframe_list[0].iloc[1: , :]
    df_b = dataframe_list[1].iloc[1: , :]
    df_e = dataframe_list[2].iloc[1: , :]
    df_f = dataframe_list[3].iloc[1: , :]
    df_h = dataframe_list[4].iloc[1: , :]

    
def m_df():
    
    # 合併 dataframe 物件
    df_all = pd.concat([df_a, df_b, df_e, df_f, df_h], axis=0, ignore_index=True)
    
def filter_b():
    
    # 計算【總件數】
    total_count = len(df_all.index)

    print('總件數: ' + str(total_count) + '件')

    
    # 計算【總車位數】
    regex = "[\u4e00-\u9fa5]{2}[0-9]{1,}[\u4e00-\u9fa5]{2}[0-9]{1,}[\u4e00-\u9fa5]{2}"

    for i in range(len(df_all.index)):
        if type(df_all['交易筆棟數'].iloc[i]) != int:
            df_all['車位數'][i] = int(re.sub(regex, '', df_all['交易筆棟數'].iloc[i]))

    total_park = df_all['車位數'].sum()
    print('總車位數: ' + str(total_park) + "個車位")


    # 計算【平均總價元】
    for i in range(len(df_all.index)):
        df_all['總價元'][i] = int(df_all['總價元'][i])
    
    total_amount = df_all['總價元'].sum()
    average_amount = round(total_amount / total_count, 2)

    print('平均總價元: ' + str(average_amount) + '元')
    
    # 計算【平均車位總價元】
    for i in range(len(df_all.index)):
        if type(df_all['車位總價元'].iloc[i]) != int:
            df_all['車位總價元'][i] = int(df_all['車位總價元'][i])

    total_park_dollar = df_all['車位總價元'].sum()
    avg_park_dollar = round(total_park_dollar / total_park, 2)

    print('平均車位總價元: ' + str(avg_park_dollar) + "元")
    
    # 整理成 filter_b dataframe
    filter_dict = {'總件數': [total_count], 
               '總車位數': [total_park], 
               '平均總價元': [average_amount], 
               '平均車位總價元': [avg_park_dollar]}

    df_filter = pd.DataFrame.from_dict(filter_dict)
    
    
    # 匯出 filter_b.csv    
    df_filter.to_csv('filter_b.csv')
    
if __name__ == "__main__":
    c_df()
    m_df()
    filter_b()

In [15]:
# 計算【總件數】
total_count = len(df_all.index)

print('總件數: ' + str(total_count) + '件')

總件數: 3850件


In [16]:
# 計算【平均總價元】
for i in range(len(df_all.index)):
    df_all['總價元'][i] = int(df_all['總價元'][i])
    
total_amount = df_all['總價元'].sum()
average_amount = round(total_amount / total_count, 2)

print('平均總價元: ' + str(average_amount) + '元')

<ipython-input-16-272cba10ff93>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['總價元'][i] = int(df_all['總價元'][i])


平均總價元: 14492507.67元


In [17]:
# 計算【總車位數】

regex = "[\u4e00-\u9fa5]{2}[0-9]{1,}[\u4e00-\u9fa5]{2}[0-9]{1,}[\u4e00-\u9fa5]{2}"

for i in range(len(df_all.index)):
    if type(df_all['交易筆棟數'].iloc[i]) != int:
        df_all['車位數'][i] = int(re.sub(regex, '', df_all['交易筆棟數'].iloc[i]))

total_park = df_all['車位數'].sum()

print('總車位數: ' + str(total_park) + "個車位")

<ipython-input-17-b7d98b3e3411>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['車位數'][i] = int(re.sub(regex, '', df_all['交易筆棟數'].iloc[i]))


總車位數: 1825個車位


In [18]:
# 計算【平均車位總價元】

for i in range(len(df_all.index)):
    if type(df_all['車位總價元'].iloc[i]) != int:
        df_all['車位總價元'][i] = int(df_all['車位總價元'][i])

total_park_dollar = df_all['車位總價元'].sum()
avg_park_dollar = round(total_park_dollar / total_park, 2)

print('平均車位總價元: ' + str(avg_park_dollar) + "元")

平均車位總價元: 1019978.62元


In [21]:
filter_dict = {'總件數': [total_count], 
               '總車位數': [total_park], 
               '平均總價元': [average_amount], 
               '平均車位總價元': [avg_park_dollar]}

df_filter = pd.DataFrame.from_dict(filter_dict)
df_filter

,總件數,總車位數,平均總價元,平均車位總價元
0,3850,1825,14492507.67,1019978.62
